<a href="https://colab.research.google.com/github/Gcarmnonapy7/Masterpiece_prototype_scheduling_movie_theaters/blob/main/movies_cluesterization_%26crowd_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import depedencies

import pandas as pd
import os
from collections import Counter

In [136]:
import kagglehub
import os
path_tmdb = kagglehub.dataset_download("tmdb/tmdb-movie-metadata")
path_movie_box = kagglehub.dataset_download("aditya126/movies-box-office-dataset-2000-2024")

Using Colab cache for faster access to the 'tmdb-movie-metadata' dataset.
Using Colab cache for faster access to the 'movies-box-office-dataset-2000-2024' dataset.


In [137]:
print(os.listdir(path_tmdb))
print(os.listdir(path_movie_box))

['tmdb_5000_movies.csv', 'tmdb_5000_credits.csv']
['enhanced_box_office_data(2000-2024)u.csv']


In [138]:
#Import datasets
import pandas as pd

tmdb = pd.read_csv(os.path.join(path_tmdb,'tmdb_5000_movies.csv'))
box = pd.read_csv(os.path.join(path_movie_box,'enhanced_box_office_data(2000-2024)u.csv'))

Predicted Crowd Week w=∑similarityi​∑(similarityi​×crowdi,w​)​

In [139]:
print(tmdb.shape)
print(box.shape)

(4803, 20)
(5000, 13)


In [177]:
box.info(
    verbose=True,
    show_counts=True
)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Rank             5000 non-null   int64  
 1   title            5000 non-null   object 
 2   worldwide_gross  5000 non-null   float64
 3   $Domestic        5000 non-null   float64
 4   Domestic %       5000 non-null   float64
 5   $Foreign         5000 non-null   float64
 6   Foreign %        5000 non-null   float64
 7   Year             5000 non-null   int64  
 8   Genres           4822 non-null   object 
 9   rating           4830 non-null   object 
 10  Vote_Count       4830 non-null   float64
 11  language         4830 non-null   object 
 12  countries        4800 non-null   object 
dtypes: float64(6), int64(2), object(5)
memory usage: 507.9+ KB


In [170]:
tmdb.info(
    verbose=True,
    show_counts=True
)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 19 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   budget                  4803 non-null   int64  
 1   genres                  4803 non-null   object 
 2   id                      4803 non-null   int64  
 3   keywords                4803 non-null   object 
 4   original_language       4803 non-null   object 
 5   original_title          4803 non-null   object 
 6   overview                4800 non-null   object 
 7   popularity              4803 non-null   float64
 8   production_companies    4803 non-null   object 
 9   production_countries    4803 non-null   object 
 10  release_date            4802 non-null   object 
 11  revenue                 4803 non-null   int64  
 12  runtime                 4801 non-null   float64
 13  title                   4803 non-null   object 
 14  vote_average            4803 non-null   

In [176]:
tmdb['keywords']

,keywords
0,"[{'id': 1463, 'name': 'culture clash'}, {'id':..."
1,"[{'id': 270, 'name': 'ocean'}, {'id': 726, 'na..."
2,"[{'id': 470, 'name': 'spy'}, {'id': 818, 'name..."
3,"[{'id': 849, 'name': 'dc comics'}, {'id': 853,..."
4,"[{'id': 818, 'name': 'based on novel'}, {'id':..."
...,...
4798,"[{'id': 5616, 'name': 'united states–mexico ba..."
4799,[]
4800,"[{'id': 248, 'name': 'date'}, {'id': 699, 'nam..."
4801,[]


In [175]:


def clean_keywords(column_keywords):

  key_words = json.loads(column_keywords)

  return key_words

tmdb['keywords'] = tmdb['keywords'].apply(clean_keywords)


tmdb['keywords']

TypeError: the JSON object must be str, bytes or bytearray, not list

In [171]:
tmdb['overview']

,overview
0,"In the 22nd century, a paraplegic Marine is di..."
1,"Captain Barbossa, long believed to be dead, ha..."
2,A cryptic message from Bond’s past sends him o...
3,Following the death of District Attorney Harve...
4,"John Carter is a war-weary, former military ca..."
...,...
4798,El Mariachi just wants to play his guitar and ...
4799,A newlywed couple's honeymoon is upended by th...
4800,"""Signed, Sealed, Delivered"" introduces a dedic..."
4801,When ambitious New York attorney Sam is sent t...


In [140]:
columns_to_rename_box_df = {'Release Group': 'title','$Worldwide':'worldwide_gross','Rating' : 'rating','Production_Countries' : 'countries','Original_Language':'language', 'Domestic':'contry_view'}

In [141]:
box.rename(columns=columns_to_rename_box_df,inplace=True) # title and total crowd around the world

In [142]:
box.columns

Index(['Rank', 'title', 'worldwide_gross', '$Domestic', 'Domestic %',
       '$Foreign', 'Foreign %', 'Year', 'Genres', 'rating', 'Vote_Count',
       'language', 'countries'],
      dtype='object')

In [169]:
# === EDA ==
import matplotlib.pyplot as plt
import seaborn as sns


sns.set_style('darkgrid')

def get_top_countries(df,column_name,top_n=10):

   df[column_name] = df[column_name].str.split(',')

   df_exploded_names = df.explode(column_name)

   df_exploded_names[column_name] = df_exploded_names[column_name].str.strip()

   return df_exploded_names[column_name].value_counts().head(top_n)

get_top_countries(box,'countries')

,count
countries,
United States of America,3153
United Kingdom,616
France,454
Japan,436
China,394
Germany,362
Canada,243
South Korea,214
Hong Kong,191


In [144]:
columns_to_drop = {'homepage','status','tagline','spoken_languages'}
tmdb = tmdb.drop(columns=columns_to_drop)

In [145]:
tmdb.info(
    verbose=True,
    show_counts=True
)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 16 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   budget                4803 non-null   int64  
 1   genres                4803 non-null   object 
 2   id                    4803 non-null   int64  
 3   keywords              4803 non-null   object 
 4   original_language     4803 non-null   object 
 5   original_title        4803 non-null   object 
 6   overview              4800 non-null   object 
 7   popularity            4803 non-null   float64
 8   production_companies  4803 non-null   object 
 9   production_countries  4803 non-null   object 
 10  release_date          4802 non-null   object 
 11  revenue               4803 non-null   int64  
 12  runtime               4801 non-null   float64
 13  title                 4803 non-null   object 
 14  vote_average          4803 non-null   float64
 15  vote_count           

In [146]:
columns_to_log = ['revenue','budget','vote_count']

def transform_log1p(data,columns_to_log):
  valid_columns = [col for col in columns_to_log if col in data.columns]
  data[valid_columns] = np.log1p(data[valid_columns])
  return data

In [147]:
# Cleaning TMDB ==> FOR FEATURE SIMILARITY

import json
import numpy as np

def clean_genres(col):
  if pd.isna(col) or col == '':
    return ''

  genres = json.loads(col)

  return ' '.join([g['name'].replace(' ','_').lower() for g in genres])

tmdb['genres'] = tmdb['genres'].apply(clean_genres)
tmdb['release_year'] = pd.to_datetime(tmdb['release_date']).dt.year


In [154]:
tmdb['genres_list'] = tmdb['genres'].apply(lambda x : x.split())
genre_exploded = tmdb.explode('genres_list')
genre_counts = genre_exploded['genres_list'].value_counts()
genre_avg_revenue = (
    genre_exploded
    .groupby('genres_list')
    ['revenue'].mean())
tmdb['genre_revenue_strength'] = tmdb['genres_list'].apply(lambda genres : np.mean([genre_avg_revenue[gen] for gen in genres] ) if len(genres) > 0 else 0)

In [158]:
tmdb[['genres_list','genre_revenue_strength']]

,genres_list,genre_revenue_strength
0,"[action, adventure, fantasy, science_fiction]",1.739210e+08
1,"[adventure, fantasy, action]",1.810758e+08
2,"[action, adventure, crime]",1.386747e+08
3,"[action, crime, drama, thriller]",8.513107e+07
4,"[action, adventure, science_fiction]",1.674433e+08
...,...,...
4798,"[action, crime, thriller]",9.613602e+07
4799,"[comedy, romance]",6.564595e+07
4800,"[comedy, drama, romance, tv_movie]",4.585203e+07
4801,[],0.000000e+00


In [164]:
print(len(tmdb[tmdb['genres_list'].apply(len) == 0]))# verify columns without genre

28


In [ ]:

#Statistical engineering

def statistical_features(df):
  pass

In [ ]:
#Outliers features

def detect_outliers(df):
  pass

In [ ]:
tmdb.head()

In [ ]:
df_final = tmdb.merge(box,on='title',how='inner')